In [ ]:
下面参考gnt-E，我们改为10层。

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

IMAGE_SIZE = 64
FULL_CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)
    
train_data = DataSetLoader(data_dir='../data/train_/')
print ('Train data loaded.')

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build 8 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns, # The input features to our model
    hidden_units=[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], # 8 hidden layers
    n_classes=CHARSET_SIZE, # survived or not {1, 0}
    model_dir="../dfs/checkpoint/dnn10_model_a", # Path to where checkpoints etc are stored
    optimizer=tf.train.RMSPropOptimizer(
        learning_rate=0.00001),
    dropout=0.1)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

classifier.train(input_fn=train_input_fn, steps=10000)
print ('Train done, begin to test ...')
test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
Train data loaded.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/dnn10_model_a', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_a/model.ckpt-40000
INFO:tensorflow:Saving checkpoints for 40001 into ../dfs/checkpoint/dnn10_model_a/model.ckpt.
INFO:tensorflow:loss = 0.0817199, step = 40001
INFO:tensorflow:global_step/sec: 0.929769
INFO:tensorflow:loss = 0.00197337, step = 40101 (107.563 sec)
INFO:tensorflow:global_step/sec: 0.644439
INFO:tensorflow:loss = 0.0112703, step = 40201 (155.180 sec)
INFO:tensorflow:global_step/sec: 0.536353
INFO:tensorflow:loss = 0.174575, step = 40301 (186.441 sec

下面是测试：

In [6]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

def make_estimator(model_dir):
    config = run_config.RunConfig(model_dir=model_dir)

    return tf.estimator.DNNClassifier (
        config=config,
        feature_columns=feature_columns, # The input features to our model
    hidden_units=[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], # 8 hidden layers
    n_classes=CHARSET_SIZE, # survived or not {1, 0}
    optimizer=tf.train.RMSPropOptimizer(
        learning_rate=0.00001),
    dropout=0.1)
    

MODEL_DIR = "../dfs/checkpoint/dnn10_model_a"
model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = model_from_checkpoint.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f023b8366d8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '../dfs/checkpoint/dnn10_model_a'}
Now processing path:  ../data/test_/00037
INFO:tensorflow:Starting evaluation at 2018-04-23-00:57:10
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_a/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2018-04-23-00:57:18
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.836199, average_loss = 1.2573

这个在2万步时，精度为80%多；在3万步时，下降了：

INFO:tensorflow:Starting evaluation at 2018-04-22-06:12:18
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_a/model.ckpt-30000
INFO:tensorflow:Finished evaluation at 2018-04-22-06:12:36
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.794118, average_loss = 1.42546, global_step = 30000, loss = 175.014

Test Accuracy: 79.411763%


再来一万步看看（14：13开始）：

INFO:tensorflow:Starting evaluation at 2018-04-23-00:57:10
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_a/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2018-04-23-00:57:18
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.836199, average_loss = 1.25737, global_step = 40000, loss = 154.377

Test Accuracy: 83.619910%

这个还真有进步！